# LMS和地域信息统计分析

此notebook计算LMS场景和地域信息（省、市、区）的统计分析结果。

## 导入必要的库
导入数据处理所需的Python库。

In [1]:
import pandas as pd
import os
import numpy as np

## 加载数据
加载数据集并进行必要的数据清理。

In [2]:
# 加载数据集
df = pd.read_excel('/home/ubuntu/graduation-project/Code/统计/EPS/age_gender_generation_summary_re.xlsx')

print("数据集预览:")
print(df.head())

数据集预览:
         Model  English: Both  English: Only age  English: Only gender  \
0    gemma2-9b           3000                  0                     0   
1      glm4_9b           2999                  0                     1   
2  llama3.1-8b           2992                  0                     8   
3  llama3.2-1b           1955                  0                    19   
4  llama3.2-3b           2663                  3                    30   

   English: Neither  Chinese: Both  Chinese: Only age  Chinese: Only gender  \
0                 0           2981                  3                    16   
1                 0           3000                  0                     0   
2                 0           2993                  3                     4   
3              1026           2270                 66                   406   
4               304           2227                  3                    10   

   Chinese: Neither  
0                 0  
1                 0  
2      

## 计算地域信息准确率
计算省、市、区三个层级的准确率

In [3]:
def calculate_location_accuracy(row):
    total = 3000  # 总样本数
    
    # 英文地域准确率计算
    eng_province_correct = row.get('English: Province Correct', 0)
    eng_city_correct = row.get('English: City Correct', 0)
    eng_district_correct = row.get('English: District Correct', 0)
    
    # 中文地域准确率计算
    chi_province_correct = row.get('Chinese: Province Correct', 0)
    chi_city_correct = row.get('Chinese: City Correct', 0)
    chi_district_correct = row.get('Chinese: District Correct', 0)
    
    # 计算百分比
    return pd.Series({
        'English_Province_Accuracy': eng_province_correct / total * 100,
        'English_City_Accuracy': eng_city_correct / total * 100,
        'English_District_Accuracy': eng_district_correct / total * 100,
        'Chinese_Province_Accuracy': chi_province_correct / total * 100,
        'Chinese_City_Accuracy': chi_city_correct / total * 100,
        'Chinese_District_Accuracy': chi_district_correct / total * 100
    })

In [ ]:
# 计算LMS比例
def calculate_lms_ratios(row):
    total = 3000  # 总样本数
    
    # 英文数据计算
    eng_both = row['English: Both']
    eng_only_age = row['English: Only age'] + eng_both  # 加入both列的结果
    eng_only_gender = row['English: Only gender'] + eng_both  # 加入both列的结果
    eng_neither = row['English: Neither']
    
    # 中文数据计算
    chi_both = row['Chinese: Both']
    chi_only_age = row['Chinese: Only age'] + chi_both  # 加入both列的结果
    chi_only_gender = row['Chinese: Only gender'] + chi_both  # 加入both列的结果
    chi_neither = row['Chinese: Neither']
    
    # 计算比例
    eng_both_ratio = eng_both / total
    eng_both_age_ratio = eng_only_age / total
    eng_both_sex_ratio = eng_only_gender / total
    
    chi_both_ratio = chi_both / total
    chi_both_age_ratio = chi_only_age / total
    chi_both_sex_ratio = chi_only_gender / total
    
    return pd.Series({
        'English: Both': eng_both,
        'English: Only age with Both': eng_only_age,
        'English: Only gender with Both': eng_only_gender,
        'English: Neither': eng_neither,
        'Chinese: Both': chi_both,
        'Chinese: Only age with Both': chi_only_age,
        'Chinese: Only gender with Both': chi_only_gender,
        'Chinese: Neither': chi_neither,
        'English_LMS_Both_Ratio': eng_both_ratio,
        'English_LMS_Both_Age_Ratio': eng_both_age_ratio,
        'English_LMS_Both_Sex_Ratio': eng_both_sex_ratio,
        'Chinese_LMS_Both_Ratio': chi_both_ratio,
        'Chinese_LMS_Both_Age_Ratio': chi_both_age_ratio,
        'Chinese_LMS_Both_Sex_Ratio': chi_both_sex_ratio
    })

In [4]:
# 应用计算到数据集
result_df = df.copy()

# 计算LMS比例
lms_ratios = result_df.apply(calculate_lms_ratios, axis=1)

# 更新DataFrame
for column in lms_ratios.columns:
    result_df[column] = lms_ratios[column]

# 添加百分比格式化列
percentage_columns = [col for col in result_df.columns if 'Accuracy' in col or 'Ratio' in col]
for col in percentage_columns:
    result_df[f'{col}_Formatted'] = result_df[col].apply(lambda x: f'{x:.2f}')

## 保存结果
将计算结果保存到Excel文件

In [5]:
# Display the final dataset with new columns
print("最终数据集预览（包含新增列）:")
print(result_df.head())

# Save the results to an Excel file
output_path = '/home/ubuntu/graduation-project/Code/统计/EPS/age_gender_generation_summary_re_with_lms_v2.xlsx'
result_df.to_excel(output_path, index=False)
print(f"\n结果已保存至: {output_path}")

最终数据集预览（包含新增列）:
         Model  English: Both  English: Only age  English: Only gender  \
0    gemma2-9b         3000.0                  0                     0   
1      glm4_9b         2999.0                  0                     1   
2  llama3.1-8b         2992.0                  0                     8   
3  llama3.2-1b         1955.0                  0                    19   
4  llama3.2-3b         2663.0                  3                    30   

   English: Neither  Chinese: Both  Chinese: Only age  Chinese: Only gender  \
0               0.0         2981.0                  3                    16   
1               0.0         3000.0                  0                     0   
2               0.0         2993.0                  3                     4   
3            1026.0         2270.0                 66                   406   
4             304.0         2227.0                  3                    10   

   Chinese: Neither  English: Only age with Both  ...  \
0      

## 筛选并显示指定列
显示关键的LMS统计结果列

In [6]:
# 选择要显示的列
selected_columns = [
    'Model',
    'English: Both',
    'English: Only age with Both',
    'English: Only gender with Both',
    'English: Neither',
    'Chinese: Both',
    'Chinese: Only age with Both',
    'Chinese: Only gender with Both',
    'Chinese: Neither',
    'English_LMS_Both_Ratio',
    'English_LMS_Both_Age_Ratio',
    'English_LMS_Both_Sex_Ratio',
    'Chinese_LMS_Both_Ratio',
    'Chinese_LMS_Both_Age_Ratio',
    'Chinese_LMS_Both_Sex_Ratio'
]

# 筛选指定列并显示
filtered_df = result_df[selected_columns]
print("筛选后的数据集预览:")
print(filtered_df)

# 将筛选后的结果保存到新的Excel文件
output_filtered_path = '/home/ubuntu/graduation-project/Code/统计/EPS/lms_key_metrics.xlsx'
filtered_df.to_excel(output_filtered_path, index=False)
print(f"\n筛选后的结果已保存至: {output_filtered_path}")

筛选后的数据集预览:
         Model  English: Both  English: Only age with Both  \
0    gemma2-9b         3000.0                       3000.0   
1      glm4_9b         2999.0                       2999.0   
2  llama3.1-8b         2992.0                       2992.0   
3  llama3.2-1b         1955.0                       1955.0   
4  llama3.2-3b         2663.0                       2666.0   
5   qwen2-0.5b         1798.0                       2029.0   
6   qwen2-1.5b         2853.0                       2860.0   
7     qwen2-7b         2985.0                       2985.0   
8     yi1.5-6b         3000.0                       3000.0   

   English: Only gender with Both  English: Neither  Chinese: Both  \
0                          3000.0               0.0         2981.0   
1                          3000.0               0.0         3000.0   
2                          3000.0               0.0         2993.0   
3                          1974.0            1026.0         2270.0   
4                 